In [1]:
import requests
page = requests.get("https://www.maxpreps.com/ca/basketball/schools/")

In [2]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(page.text,'html.parser')

In [3]:
target_text = soup.find(string="California Boys Basketball High School Teams") #returns navigable string so we gotta gop to parent
curr_elem = target_text.find_parent()
tags = []
while len(tags)==0 and curr_elem.name != '[document]': 
    search_set = [curr_elem] + curr_elem.find_next_siblings()
    for item in search_set:
        found_links = item.find_all('a') #find all anchor tags 
        if found_links:
            tags.extend(found_links) 
    curr_elem = curr_elem.parent

In [5]:
links = [tag['href'] for tag in tags if 'href' in tag.attrs] #each element is an object and *.attr is a dictionary


In [36]:
links_hsball = [link for link in links if link.startswith("/ca/") and link.endswith("/basketball/") and link != "/ca/basketball/"] #[expression for item in iterable if condition]
#each element is an object and *.attr is a dictionary
#also on MAXPREPS it is state/county/team/basketball

In [34]:
print(len(links_hsball)) #FUNCTIONAL TO HERE also, 200 teams currently in CA as of Feb 17 2026

200


In [37]:
page = requests.get("https://www.maxpreps.com/SC/basketball/schools/")
soup = BeautifulSoup(page.text,'html.parser')

In [38]:
target_text = soup.find(string="South Carolina Boys Basketball High School Teams") #returns navigable string so we gotta gop to parent
curr_elem = target_text.find_parent()
tags = []
while len(tags)==0 and curr_elem.name != '[document]': 
    search_set = [curr_elem] + curr_elem.find_next_siblings()
    for item in search_set:
        found_links = item.find_all('a') #find all anchor tags 
        if found_links:
            tags.extend(found_links) 
    curr_elem = curr_elem.parent

In [39]:
links = [tag['href'] for tag in tags if 'href' in tag.attrs] #each element is an object and *.attr is a dictionary


In [87]:
page = requests.get("https://www.maxpreps.com/ca/basketball/schools/")
soup = BeautifulSoup(page.text,'html.parser')

In [72]:
target_text = soup.find_all(['h1','h2','h3'])
target_text[-1].get_text()
target_text #WIP on pinpointing exact div-- for the far future


[<h3 class="heading_100_bold">POPULAR SPORTS</h3>,
 <h3 class="sc-9bfabda6-1 iKDzCD heading_100_bold">Apps</h3>,
 <h3 class="sc-9bfabda6-1 iKDzCD heading_100_bold">Pro Photography</h3>,
 <h3 class="sc-9bfabda6-1 iKDzCD heading_100_bold">Watch</h3>,
 <h3 class="sc-9bfabda6-1 iKDzCD heading_100_bold">Content Contributors</h3>,
 <h3 class="sc-9bfabda6-1 iKDzCD heading_100_bold">Tickets</h3>,
 <h1 class="sc-e96201a1-0 iItjPn" title="South Carolina Boys Basketball High School Teams">South Carolina Boys Basketball High School Teams</h1>]

In [136]:
import json 
import re
from bs4 import BeautifulSoup
import requests

state_dict = {"ca":"California","sc":"South Carolina"} #Set up dictionary that matches state initials to state name
sport_set = {"basketball","baseball","football"} #Set up sports set 
redundant_words = {"maxpreps","http","https","www","com"}
DEV = 1

relevant_words = {}

#IN USE NOW
def find_node(html,title = ""):
    if title == "":
        print("NOT YET READY")
        return 0
        #attempt to find title but for rn we hardcode it
        #TODO
        
    target_text = html.find(string=title)
    #returns navigable string so we gotta go to parent node
    #can set up here with predetermined syntax via json lookup or guess. go with json for now.
    return target_text.find_parent()

def grab_links(page_elem):
    tags = []
    curr_elem = page_elem
    while len(tags)==0 and curr_elem.name != ['document']: 
        search_set = [curr_elem] + curr_elem.find_next_siblings()
        for item in search_set:
            found_links = item.find_all('a') #find all anchor tags 
            if found_links:
                tags.extend(found_links) 
        curr_elem = curr_elem.parent
    links = [tag['href'] for tag in tags if 'href' in tag.attrs] #find links
    return links

    
#IN USE NOW
def grab_team_links(teams_node):
    links = grab_links(teams_node)

    ####REWORK vvvvv
    team_links = [link for link in links if link.startswith("/ca/") and link.endswith("/basketball/") and link != "/ca/basketball/"]
    ####HARDCODED RN but can use relevant words from parse_url_context in the future to decipher relevant links
    return team_links

def get_html(URL): #gets html in text
    page = requests.get(URL) 
    soup = BeautifulSoup(page.text,'html.parser') #take html
    return soup

#WIP
def parse_url_context(URL, redundant_words): # remove maxpreps and https:// etc but keep keywords like ca
    # sc, Basketball, Boys to use and find relevant themes. This way, we can remove redundant words from links within page that may point to itself
    #can use these words and additional context to generate likelihood of teams div
    #not use right now
    #chop up url
    url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
    return url_words

#WIP
def get_team_URL(page_html):
    #try to guess which link is the teams URL on a given site
    return

In [107]:
print(len(grab_team_links(team_node)))

200


In [127]:

URL = "https://www.maxpreps.com/ca/basketball/schools/"
with open("Resources/SiteInfo.json", "r") as f:
    site_rules = json.load(f)
maxprep_set = site_rules['maxpreps'] 
dev_set = maxprep_set['CA_DEV']
team_node = find_teams_node(dev_set['title'])
#print(grab_team_links(team_node))

In [65]:
redundant_words = {"maxpreps","http","https","www","com"}
URL = "https://www.maxpreps.com/ca/basketball/schools/"
url_words = [w for w in re.split(r'[/\.:]+',URL) if w.lower() not in redundant_words and len(w) > 0]
url_words

['ca', 'basketball', 'schools']

In [140]:
####PLAYER LOGIC FROM ROSTER#####

#given a team, find the roster link <-- skip since we hardcode it rn

#TODO: from rosters page, find player links

def grab_player_links(roster_element):
    links = grab_links(roster_element)

roster_elem = find_node(laf_ball_roster_html,"Acalanes  Basketball Roster")

<h1 class="title">Acalanes  Basketball Roster</h1>

In [102]:
laf_ball_roster_html = get_html("https://www.maxpreps.com/ca/lafayette/acalanes-dons/basketball/roster/")

In [134]:
roster_element = laf_ball_roster_html.find(string = "Acalanes  Basketball Roster").find_parent()
links = grab_links(roster_element)
links

['/ca/lafayette/acalanes-dons/basketball/roster/',
 '/ca/lafayette/acalanes-dons/basketball/staff/',
 '/ca/lafayette/acalanes-dons/athletes/preston-hilsabeck/?careerid=hpb7k1kir1dm6',
 '/ca/lafayette/acalanes-dons/athletes/cameron-hood/?careerid=dsg8ao87v6ep9',
 '/ca/lafayette/acalanes-dons/athletes/julian-hood/?careerid=56mf96hmdg7o8',
 '/ca/lafayette/acalanes-dons/athletes/justice-hembrador/?careerid=2kfkn0i2o19bb',
 '/ca/lafayette/acalanes-dons/athletes/sam-phillips/?careerid=qd4gfkvejm577',
 '/ca/lafayette/acalanes-dons/athletes/bryce-mansour/?careerid=hblf6vc03u0bb',
 '/ca/lafayette/acalanes-dons/athletes/aj-hastings/?careerid=5s34l10a9ftee',
 '/ca/lafayette/acalanes-dons/athletes/evan-palmer/?careerid=e40dlmenhoaaa',
 '/ca/lafayette/acalanes-dons/athletes/shea-stahl/?careerid=d8rohq05aqrvf',
 '/ca/lafayette/acalanes-dons/athletes/jon-macleod/?careerid=23f38c690m8td',
 '/ca/lafayette/acalanes-dons/athletes/gavin-dodge/?careerid=2eorqhnhgcb44',
 '/ca/lafayette/acalanes-dons/athlete

In [ ]:
#TODO
    #continue on roster directory logic
    #continue on player page logic
    #make it create json file that maps teams(from DB) to links
        #"data_pipeline_shortcuts":{
        #"team_url_pattern": "",
        #"team_page_url_pattern": "",
        #"roster_page_url_pattern": "",
        #"player_page_url_pattern": ""
        #},